In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import pandas as pd

2026-02-06 09:02:35.962717: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


kaggle datasets download -d mirzaniazmorshed/ntsb-aviation-accidents
unzip ntsb-aviation-accidents.zip -d ntsb_data

In [2]:

df_narr = pd.read_excel('ntsb_data/narratives.xlsx')
df_find = pd.read_excel('ntsb_data/Findings_merged.xlsx')
df_joined = pd.merge(df_narr, df_find, on='ev_id', how='inner')
df = df_joined[['narr_accp', 'finding_description']].dropna()

df['broad_cause'] = df['finding_description'].str.split('-').str[0]
df = df[df.groupby('broad_cause')['broad_cause'].transform('count') > 50]
df['broad_cause'].value_counts()

broad_cause
Personnel issues         24358
Aircraft                 21886
Environmental issues     11838
Not determined            2134
Organizational issues      709
Name: count, dtype: int64

In [7]:
import tensorflow as tf
import keras_nlp
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import class_weight

tf.keras.mixed_precision.set_global_policy("mixed_float16")

valid_classes = df['broad_cause'].value_counts()[df['broad_cause'].value_counts() > 3000].index
df = df[df['broad_cause'].isin(valid_classes)].copy()

encoder = LabelEncoder()
df['label'] = encoder.fit_transform(df['broad_cause'])

X = df['narr_accp'].astype(str).to_numpy()
y = df['label'].to_numpy()

X_train, X_val, y_train, y_val = train_test_split(
    X, 
    y, 
    test_size=0.2, 
    stratify=y  
)

weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(weights))

classifier = keras_nlp.models.BertClassifier.from_preset(
    "bert_base_en_uncased",
    num_classes=num_classes,
    load_weights=True,
    activation="softmax"
)

classifier.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    metrics=["accuracy"]
)

classifier.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size=32,      
    epochs=4,           
    class_weight=class_weight_dict
)

Epoch 1/4


2026-02-06 09:11:10.104957: I external/local_xla/xla/service/gpu/autotuning/dot_search_space.cc:208] All configs were filtered out because none of them sufficiently match the hints. Maybe the hints set does not contain a good representative set of valid configs? Working around this by using the full hints set instead.
2026-02-06 09:11:10.611956: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_253', 8 bytes spill stores, 8 bytes spill loads

2026-02-06 09:11:11.196973: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_278', 4 bytes spill stores, 4 bytes spill loads

2026-02-06 09:11:12.113310: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_76', 8 bytes spill

1452/1453 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.4188 - loss: 1.0440

2026-02-06 09:17:43.989340: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_367', 16 bytes spill stores, 16 bytes spill loads

2026-02-06 09:17:44.747808: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_291', 16 bytes spill stores, 16 bytes spill loads

2026-02-06 09:17:44.774672: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_283', 16 bytes spill stores, 16 bytes spill loads

2026-02-06 09:17:44.969913: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_175', 16 bytes spill stores, 16 bytes spill loads



1453/1453 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step - accuracy: 0.4189 - loss: 1.0440

2026-02-06 09:18:37.149178: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_5', 1260 bytes spill stores, 1260 bytes spill loads



1453/1453 ━━━━━━━━━━━━━━━━━━━━ 480s 291ms/step - accuracy: 0.4466 - loss: 1.0085 - val_accuracy: 0.4654 - val_loss: 0.9787
Epoch 2/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 390s 268ms/step - accuracy: 0.4807 - loss: 0.9471 - val_accuracy: 0.4643 - val_loss: 0.9996
Epoch 3/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 389s 268ms/step - accuracy: 0.4947 - loss: 0.9087 - val_accuracy: 0.4487 - val_loss: 1.0242
Epoch 4/4
1453/1453 ━━━━━━━━━━━━━━━━━━━━ 389s 268ms/step - accuracy: 0.5104 - loss: 0.8699 - val_accuracy: 0.4493 - val_loss: 1.0916
